In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


In [3]:
from IPython.display import display, HTML

# 设置默认输出折叠
display(HTML('''
<style>
.output {
    display: flex;
    flex-direction: column;
}
.output_scroll {
    max-height: 300px; 
    overflow-y: auto;
}
</style>
'''))

## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [4]:
x_test_original, y_test_original,x_train, y_train = ReadTS('ChlorineConcentration')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)

(3840, 166, 1)


In [5]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    # 获取原始数据集的形状
    original_shape = TS.shape
    
    # 将数据集reshape成二维数组
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    # 初始化StandardScaler
    scaler = StandardScaler()
    
    # 对数据集进行标准化
    TS_normalized = scaler.fit_transform(TS_2d)
    
    # 将标准化后的数据集reshape回原始形状
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [6]:
model = make_CNN_model(x_train.shape[1:],num_classes = 3)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=150, batch_size=32)

Epoch 1/150


2024-11-30 11:04:35.469172: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


96/96 [==============================] - 1s 11ms/step - loss: 0.9850 - sparse_categorical_accuracy: 0.5573 - val_loss: 1.0674 - val_sparse_categorical_accuracy: 0.5352
Epoch 2/150
96/96 [==============================] - 1s 10ms/step - loss: 0.9580 - sparse_categorical_accuracy: 0.5687 - val_loss: 1.0677 - val_sparse_categorical_accuracy: 0.4935
Epoch 3/150
96/96 [==============================] - 1s 10ms/step - loss: 0.9607 - sparse_categorical_accuracy: 0.5729 - val_loss: 1.0442 - val_sparse_categorical_accuracy: 0.5508
Epoch 4/150
96/96 [==============================] - 1s 11ms/step - loss: 0.9482 - sparse_categorical_accuracy: 0.5736 - val_loss: 0.9764 - val_sparse_categorical_accuracy: 0.5664
Epoch 5/150
96/96 [==============================] - 1s 11ms/step - loss: 0.9467 - sparse_categorical_accuracy: 0.5723 - val_loss: 0.9813 - val_sparse_categorical_accuracy: 0.5846
Epoch 6/150
96/96 [==============================] - 1s 11ms/step - loss: 0.9404 - sparse_categorical_accuracy: 

In [7]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

15/15 [==============================] - 0s 3ms/step - loss: 1.8179 - sparse_categorical_accuracy: 0.6445
Test accuracy 0.6445395946502686
Test loss 1.817866563796997


### Create perturbed test sets and calculate decrease in accuracy

In [8]:
print(y_test_original.shape)

(467,)


In [31]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 100
index_1 = 160

x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

pert1 = perturb('RBP1',perturb_test_set[0],index_0,index_1, global_ts = perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1',x,index_0,index_1, global_ts = perturb_test_set) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

15/15 [==============================] - 0s 2ms/step


In [32]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

def AveragSuccessRate(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] != labels[i]:
            hits += 1
    return hits/len(predictions)

In [33]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

7.077537295635214
7.564985445557915
7.538781168056821
9.88836843827624
3.623919762419179


In [34]:
AveragSuccessRate(np.argmax(predictions_new1,axis=1), np.argmax(predictions_original,axis=1))

0.11777301927194861

In [35]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [36]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.6445396145610278
 RBP Accuracy: 0.588865096359743
 RBP1 Accuracy: 0.5610278372591007
 Zero Accuracy: 0.5717344753747323
 Noise Accuracy: 0.6038543897216274
 Blur Accuracy: 0.576017130620985

